In [4]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [5]:
import os
from Bio import SeqIO
import subprocess

family_group = "10.Lycaenidae"

output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Vanessa_tameamea"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            error_exons += coordinate_file_list[i]
        
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "24":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Aricia_agestis
GCF_905147365.1_ilAriAges1.1_genomic.fna
33839821 33839909
Getting Gene
MEIVEDTEKRSGVADSAYSYSCSNSKSTQ
>Aricia_agestis_Exon_1_NC_056428.1_33839821_33839909_left_0_right_2
ATGGAGATTGTTGAAGATACAGAGAAACGTAGCGGAGTTGCTGACTCCGCTTATTCGTACAGTTGCAGCAACAGTAAATCTACTCAAAG
33826718 33826792
Getting Gene
>Aricia_agestis_Exon_2_NC_056428.1_33826718_33826792_left_1_right_2
TTCAAAATCTACGTCAGAGAGTAACTCGAGCGGCAGCAGTGGCTACCGAGGGAAACGCACTGATGCTGATTACAT
33826164 33826336
Getting Gene
>Aricia_agestis_Exon_3_NC_056428.1_33826164_33826336_left_1_right_1
CAAAAACAATAATGCACAACCTATTATAGAGAAAAGAGTTAATGATGAGATCAAAACTGCGGAAGTTAAACTAAAAAGCGCAGAAGTTGTTAATAATCATGATGCACAAACTATAGAATCTAAAGATACATCACCACCAACAACAGAAGATGTTAAAGATGGATCTGACAATG
33824396 33824493
Getting Gene
>Aricia_agestis_Exon_4_NC_056428.1_33824396_33824493_left_2_right_0
TTTACCTTAGTTCTCCACCGCCTACGGTTGGAGAAGGGGGCAATGAGTTAGATAACATGGACATATCTACGTCAGACAAAAGCACACGACAAAATAAA
33823892 33823999
Getting Gene
>Aricia_agestis_Exon_5_NC_056428.1_33823892_3382

>Aricia_artaxerxes_Exon_14_OW569310.1_36530663_36530746_left_1_right_2
ATACTTCGATAGCCATGAGACATGCATCGTCAATAGTTATACCACCGATTACGCGTTCCAGCCGCAACCGTCCACTTCCAAGGC
36530174 36530305
Getting Gene
>Aricia_artaxerxes_Exon_15_OW569310.1_36530174_36530305_left_1_right_2
AGTCGCACCCACTACAAACGCGAAAGTATCGCCTATACCGTCTGAGCAGACGACGACATCTACAAGCTCAAGCCCTAACTCAACTgtattacaaaacaaaactgTTTGGCTAACAGAATCGTTGCTAAACAA
36529229 36529394
Getting Gene
>Aricia_artaxerxes_Exon_16_OW569310.1_36529229_36529394_left_1_right_0
gCATAACGAAAAAATGGAAAAGAAATTGGTTAAAACGCACTACAAGAATAAATCTTTGATGAAAGGAGAAAGGGGAAAGTCGAAAGAGATAAAAATTACCAAACAGAAGAACAACGAAATTTTGGACAGGTGCAACAAGCCCACAGCAGGCGACCATTTGGCTCAT
36527495 36527624
Getting Gene
>Aricia_artaxerxes_Exon_17_OW569310.1_36527495_36527624_left_0_right_1
TTGAATGGTTTAAAGCGTCCTACGGACTCTATGAGCTGTCCTATTCCATGCAAAACTCGCTGCACTGTTGTTAACGGGGATCGAACAAAATCCGTTGCAATGGTGCCTGTTTTCCCTAATGCTGCTATTG
36527013 36527129
Getting Gene
>Aricia_artaxerxes_Exon_18_OW569310.1_36527013_36527129_left_2_right_1
CACCC

15109993 15110067
Getting Gene
>Celastrina_argiolus_Exon_2_LR994583.1_15109993_15110067_left_1_right_2
TACAAAGTCTACGTCGGAGAGCAACTCTAGCGGCAGCAGTGGATATAGAGGAAAAAGGGCTGATGCCGATTACAT
15105611 15105783
Getting Gene
>Celastrina_argiolus_Exon_3_LR994583.1_15105611_15105783_left_1_right_1
AAAAACCACTAATGCACTACCTACTTTGGAGAAAAGAGTTAATGATGAGACGAAGGCGGTGGATATAAAGCTAAAATGCGCAGAAGTTGTAAATAATCATGACGCACAAACTGCAGAATTAAAAGCTGCATCCCCATCTACCAACGAGGAGGCTAAAGATGGAATTGACAATG
15102966 15103063
Getting Gene
>Celastrina_argiolus_Exon_4_LR994583.1_15102966_15103063_left_2_right_0
TTTACCTTAGTTCTCCCCCGGCCACAGTGGGAGAGATGGGGAAAGAATCGGATAGCATGGATATATCCACATCAGAAAAAAGCACAAGCCAAAATAAA
15101871 15101978
Getting Gene
>Celastrina_argiolus_Exon_5_LR994583.1_15101871_15101978_left_0_right_0
GATTTGGTAATCTGTATCAAGACTAAGGATTATTTCCAGAGTACAGTGTGCGACCCGCACCCTTCTTCTATCATATGCACCATAGACTATCCCAATACACATTGTTCT
15091018 15091149
Getting Gene
>Celastrina_argiolus_Exon_6_LR994583.1_15091018_15091149_left_0_right_0
GATGGATTCTTTTGCGTTATGTCAATG

15925412 15925524
Getting Gene
>Cyaniris_semiargus_Exon_13_LR994547.1_15925412_15925524_left_0_right_2
aaaactGGCTCGATCAATCTCGCTGGTATATTATCACAGTGTACCGAGAGCAAGTCGAGCACAGACACATCTATTAGCTACGATCGACTCAACTACGATGAGACTTTGCAGAG
15924521 15924604
Getting Gene
>Cyaniris_semiargus_Exon_14_LR994547.1_15924521_15924604_left_1_right_2
ATACTTCGATAGCCACGAGCCATGCATCGTCAATAGTTATACCACCGATTACGCGTTCCATCCGCAACCGTCCACTTCCAAGGC
15924018 15924149
Getting Gene
>Cyaniris_semiargus_Exon_15_LR994547.1_15924018_15924149_left_1_right_2
AGGCGCACCCACTACAAGCGTGAAAGTATCGCCTATAGCGTCTGAGCAGACCACGGCATCAACAAGCTCAAGCCCTAACTCAActgttttacaaaataaaactgtTCGGCTAACAGAATCGTTGCTAAACAA
15923054 15923228
Getting Gene
>Cyaniris_semiargus_Exon_16_LR994547.1_15923054_15923228_left_1_right_0
GCATAACGAAAAAATGGAAAAGAAATTGGTTAAAACGCACTACAAGAATAAATCTTTGATCAAGGAAGAAAGGGGAAAGTTGAAAGAGATAAAAATAACCAAACAGAAGATGAACGAACATTTGGACAGGTGCAACGAGTCATACCAGCCCACAGCAGGCGACCATTTGGCTCAT
15921734 15921863
Getting Gene
>Cyaniris_semiargus_Exon_17_LR994547.1_15921734_1

9972878 9972966
Getting Gene
MEIVEETEKRSGVGDSAYSYSCSNSKSTQ
>Lysandra_bellargus_Exon_1_HG995319.1_9972878_9972966_left_0_right_2
ATGGAGATAGTTGAAGAGACGGAGAAACGTAGCGGAGTTGGTGACTCCGCATATTCGTATAGCTGCAGCAACAGTAAATCTACTCAAAG
9976923 9976997
Getting Gene
>Lysandra_bellargus_Exon_2_HG995319.1_9976923_9976997_left_1_right_2
TACAAAATCTACGTCAGAGAGTAACTCGAGCGGTAGCAGTGGCTACAGAGGGAAACGCACCGATGCCGATTACAT
9977411 9977586
Getting Gene
>Lysandra_bellargus_Exon_3_HG995319.1_9977411_9977586_left_1_right_1
CAAAAACAATAATGCACAACCTAATGTAGAGAAAAGAGTTAATGATGAGACGAAAACTGTGGATGTTAAACTAAAAAGCTCAGAAGGTGTCAATGATCATGATGCACAAACTGTAGAATTTAAAGATACATCACCATCAACAACAACTGATGAAGTTAAAGTTGGATCTGACAATG
9979516 9979610
Getting Gene
>Lysandra_bellargus_Exon_4_HG995319.1_9979516_9979610_left_2_right_0
TTTACCTTAGTTCTCCACCGCCCACGGTTGGAGAGAGGAACAAAGAGTCAGATAGCATGGACATATCCACGTCAGACAAAAACACACGAAAAAAA
9980167 9980274
Getting Gene
>Lysandra_bellargus_Exon_5_HG995319.1_9980167_9980274_left_0_right_0
GACCTGGAATTCTGTGTTAAGACGAAGGACTCTTTCCAGAG

23649467 23649586
Getting Gene
>Lysandra_coridon_Exon_11_HG992055.1_23649467_23649586_left_0_right_0
ggccCAACAAATCCGAACGTGTTTGATTCGCCGGAGGAAGTAAGAACTGCTAAATTTAGTGAAGAAGAAATAATTAAAAGACTGCAACTGACCAACAGCATTAAGAAACTTTTGAATgaa
23648324 23648455
Getting Gene
>Lysandra_coridon_Exon_12_HG992055.1_23648324_23648455_left_0_right_0
GTGTTATCAAAACCAACGAAATTGGCCAAACAGCAGATGACTAAAAGATGTCAGGAGATCGCTGCGTTTATGGAGACATTCGTAGGCAAGATCACCGTTGATCCTTGTATACCGTTTAAAAATACCAAATGT
23647608 23647723
Getting Gene
>Lysandra_coridon_Exon_13_HG992055.1_23647608_23647723_left_0_right_2
aaAAGTGACTCAATCAAACTCGAAGGTATATTATCACAGTGTACCGAATGCAATAATTCGAGCACAGACACATCCATAAGCTACGATCGACTCAACTATGATGAGACTTTGCAGAG
23646157 23646240
Getting Gene
>Lysandra_coridon_Exon_14_HG992055.1_23646157_23646240_left_1_right_2
ATACTTCGATAGCCATGAGCCATGCATCGTCAATAGTTATACCACCGATTACGCGTTCCAGCCGCAATCGTCCACTCCCATGGC
23645666 23645797
Getting Gene
>Lysandra_coridon_Exon_15_HG992055.1_23645666_23645797_left_1_right_2
AGTCGCATCCACAACAAACGCGAAAGCATCGCCTCTATC

16871767 16871904
Getting Gene
>Phengaris_arion_Exon_23_OY751437.1_16871767_16871904_left_0_right_0
CCGGCCTTAGTGAATGAGCAGCTAGATCAGCTGTACATGGACCTGCAACTCGAGGGTGTGGCGGCGAGACTCACTTTAGAGGAGGGAATCACCAGTTCCGGCAGCTCGTCGGACGAAACCTCCGCGCACTGCTCCAAG
16867821 16867934
Getting Gene
>Phengaris_arion_Exon_24_OY751437.1_16867821_16867934_left_0_right_0
AACCGCAAACCCATTCACCGTCGTCGGCGAGGCTACGACAAAATGGTGATGCTGTATGAAGACGACGCGCCCCTCCCACCCCCAGAAGACACCGCATCCTCCGTAGCGACGTAA
Plebejus_argus
GCA_905404155.3_ilPleArgu1.3_genomic.fna
6741526 6741614
Getting Gene
MEVVEETEKRSGVGDSAYSYSCSNSKSTQ
>Plebejus_argus_Exon_1_FR989932.1_6741526_6741614_left_0_right_2
ATGGAGGTTGTTGAAGAGACGGAGAAACGTAGCGGAGTTGGTGACTCCGCGTATTCGTATAGCTGCAGCAACAGCAAATCTACTCAAAG
6745455 6745529
Getting Gene
>Plebejus_argus_Exon_2_FR989932.1_6745455_6745529_left_1_right_2
TTCAAAATCTACGTCAGAGAGTAACTCGAGCGGCAGCAGTGGCTACAGAGGGAAACGCACTGAGGCTGAATACAT
6745979 6746151
Getting Gene
>Plebejus_argus_Exon_3_FR989932.1_6745979_6746151_left_1_right_1
AAAAAACAATAA

30187889 30188008
Getting Gene
>Polyommatus_icarus_Exon_11_OW569320.1_30187889_30188008_left_0_right_0
GGCCCCACAAATCCCAACGTGTTTGATATGCCAGAGGAAGAAAAATCTGCCAAATTGAATGAAAAAGAAAGAGCTAAAATACAACAAATGACCAACAGCATTAAGAAACTATTAAATgaa
30187416 30187547
Getting Gene
>Polyommatus_icarus_Exon_12_OW569320.1_30187416_30187547_left_0_right_0
GTGTTATCAAAACCAACGAAATTGGCCAAACAGCAGATGACTAAAAGATGTCAGGAAATTGCTGCCTTTATGGAGTCGTTCATAGACAAAACCAACATTAATAGTGCTATGCCGTTTAAAGATAAGAAATGT
30186910 30187022
Getting Gene
>Polyommatus_icarus_Exon_13_OW569320.1_30186910_30187022_left_0_right_2
gAAACTGACTCAATCAAACTTGGAGGTATATTATCACAGTGTACCGACAGCAAGTCGAGCACAAGCACATCTATTAGCTACGATCGACTCAACTACGATGAGACTCTTCAGag
30186415 30186498
Getting Gene
>Polyommatus_icarus_Exon_14_OW569320.1_30186415_30186498_left_1_right_2
ATACTTCGATAGCCATGAGCCACTTATCGTCAATAGTTATACCACCGGTTACGCGTTCCAGCCGCAACCGTCCACTTCCAAGGG
30185981 30186112
Getting Gene
>Polyommatus_icarus_Exon_15_OW569320.1_30185981_30186112_left_1_right_2
AGTCGCACCCACTACAAACGCCAAAGTATCGC

31354086 31354223
Getting Gene
>Polyommatus_iphigenia_Exon_23_OY730178.1_31354086_31354223_left_0_right_0
CCGTCCCTAGTCAACGAGCAGCTCGGCCAGCTATACATGGACATGCAGCTGGAGGGTGTGGCTGCCAGGCTGACCCTGGAGGAGGGCATCACCAGCTCCGGGAGCTCATCGGATGAGAATACCGCACACGCCTCCACG
31352945 31353055
Getting Gene
>Polyommatus_iphigenia_Exon_24_OY730178.1_31352945_31353055_left_0_right_0
TCCAAGAAAACCGTGAAACGTCGTCATCGAGGCTACGATAAGATGGTTATAGTGTACGAAGAAGAcgcccccctccccccaccccaaGAAACAGAGACGGCCACGACGTAG

























